In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import random as rand
from selenium.webdriver.common.action_chains import ActionChains
import pickle
import datetime
from fake_useragent import UserAgent
from configparser import ConfigParser

# Random

In [2]:
def random_sleep(start, end):
    time.sleep(rand.randint(start, end))

def random(start,end):
    return rand.randint(start, end)

# Driver

In [3]:
def createDriver():
    ua = UserAgent()
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument(f'--user-agent={ua.random}')
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--incognito")
    options.add_argument("--disable-popup-blocking")
    driver = webdriver.Chrome(options)
    return driver

# Login

In [4]:
def login(driver):
    try:
        config = ConfigParser()
        config.read('config.ini')
        USERNAME = config['X']['username']
        PASSWORD = config['X']['password']
        driver.get("https://x.com/login")
        random_sleep(2,7)
        username = driver.find_element(By.XPATH, '//*[@id="layers"]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/div[4]/label/div/div[2]/div/input')
        username.send_keys(USERNAME)
        random_sleep(2,7)
        next = driver.find_element(By.XPATH, '//*[@id="layers"]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/button[2]/div')
        next.click()    
        random_sleep(2,7)
        password = driver.find_element(By.XPATH, '//*[@id="layers"]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[1]/div/div/div[3]/div/label/div/div[2]/div[1]/input')
        password.send_keys(PASSWORD)
        random_sleep(2,7)
        login = driver.find_element(By.XPATH, '//*[@id="layers"]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div[1]/div/div/button')
        login.click()
        random_sleep(2,7)
        return driver
    except Exception as e:
        print(f"Login failed: {str(e)}")
        return None


# Search

In [5]:
def search(driver,searchStr="#deprem lang:tr until:2023-02-07 since:2023-02-06"):
    search = driver.find_element(By.XPATH, '//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div[2]/div/div[2]/div/div/div/div[1]/div/div/div/form/div[1]/div/div/div/div/div[2]/div/input')
    search.send_keys(searchStr)
    search.send_keys(Keys.ENTER)
    random_sleep(5,9)
    latest = driver.find_element(By.XPATH, '//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div[1]/div/div[1]/div[1]/div[2]/nav/div/div[2]/div/div[2]/a')
    latest.click()
    random_sleep(5,9)
    return driver

# Random Scroll Tweets

In [12]:
def random_scroll_tweets(driver, element):
    scroll_amount = rand.randint(1000, 3000)
    scroll_duration = rand.uniform(0.5, 2)
    try:
        driver.execute_script(
            "arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", 
            element
        )
        driver.execute_script(
            f"window.scrollBy(0, {scroll_amount});", 
        )
        time.sleep(scroll_duration)
    except Exception as e:
        print(f"Scroll failed: {str(e)}")

# Random Scroll

In [13]:
def smooth_random_scroll(driver):
    scroll_amount = rand.randint(1000, 2000)
    scroll_duration = rand.uniform(0.5, 2)
    try:
        driver.execute_script(
            f"window.scrollBy({{ top: {scroll_amount}, behavior: 'smooth' }});"
        )
        time.sleep(scroll_duration)
    except Exception as e:
        print(f"Smooth scroll failed: {str(e)}")

# Random Click

In [14]:
xpath_list = [
    '//*[@id="react-root"]/div/div/div[2]/header/div/div/div/div[1]/div[2]/nav/a[2]',  # Explore
    '//*[@id="react-root"]/div/div/div[2]/header/div/div/div/div[1]/div[2]/nav/a[3]',  # Notifications
    '//*[@id="react-root"]/div/div/div[2]/header/div/div/div/div[1]/div[2]/nav/a[4]',  # Messages
    '//*[@id="react-root"]/div/div/div[2]/header/div/div/div/div[1]/div[2]/nav/a[5]',  # Grok
    '//*[@id="react-root"]/div/div/div[2]/header/div/div/div/div[1]/div[2]/nav/a[6]',  # Lists
    '//*[@id="react-root"]/div/div/div[2]/header/div/div/div/div[1]/div[2]/nav/a[7]',  # Bookmarks
    '//*[@id="react-root"]/div/div/div[2]/header/div/div/div/div[1]/div[2]/nav/a[12]'  # Profile
]

def random_click(driver):
    xpath = rand.choice(xpath_list)
    try:
        element = driver.find_element(By.XPATH, xpath)
        element.click()
        random_sleep(2,7)        
        driver.back()
        random_sleep(2,7)
        
    except Exception as e:
        print(f"Error while clicking: {str(e)}")

# GetData

In [15]:
def get_tweets(driver,file="tweets10.txt"):
    seen_tweets = set()
    
    with open(file, "a", encoding="utf-8") as file:
        while True:
            timeline_divs = driver.find_elements(By.XPATH, '//div[@data-testid="cellInnerDiv"]')
            for div in timeline_divs:
                try:
                    tweet_text = div.text.strip() if div.text else ""
                    if tweet_text and tweet_text not in seen_tweets:
                        seen_tweets.add(tweet_text)
                        file.write(f"Timestamp: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\nText: {tweet_text}\n\n")
                
                except Exception as e:
                    print(f"An error occurred while processing the tweet: {str(e)}")
                    continue  
            
            if timeline_divs:
                random_scroll_tweets(driver, timeline_divs[-1])
                random_sleep(1, 2)
                for _ in range(rand.randint(1, 2)):
                    random_click(driver)
                    smooth_random_scroll(driver)
                
            #ActionChains(driver).move_to_element(timeline_divs[-1]).perform()
            #random_sleep(1,2)

In [18]:
driver = createDriver()
driver = login(driver)
random_sleep(3,7)
driver = search(driver,"#deprem lang:tr until:2023-02-07 since:2023-02-05")
get_tweets(driver)
driver.quit()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.69)
Stacktrace:
	GetHandleVerifier [0x00007FF7AC8D2775+28773]
	(No symbol) [0x00007FF7AC83AFB0]
	(No symbol) [0x00007FF7AC6D552A]
	(No symbol) [0x00007FF7AC6AF2B5]
	(No symbol) [0x00007FF7AC755FD7]
	(No symbol) [0x00007FF7AC76EA92]
	(No symbol) [0x00007FF7AC74ED43]
	(No symbol) [0x00007FF7AC71A548]
	(No symbol) [0x00007FF7AC71B6B1]
	GetHandleVerifier [0x00007FF7ACBFF45D+3358029]
	GetHandleVerifier [0x00007FF7ACC1430D+3443709]
	GetHandleVerifier [0x00007FF7ACC083FD+3394797]
	GetHandleVerifier [0x00007FF7AC99929B+842635]
	(No symbol) [0x00007FF7AC84654F]
	(No symbol) [0x00007FF7AC841FA4]
	(No symbol) [0x00007FF7AC84213D]
	(No symbol) [0x00007FF7AC831629]
	BaseThreadInitThunk [0x00007FFC4BE3259D+29]
	RtlUserThreadStart [0x00007FFC4DD2AF38+40]
